In [24]:
import glob
import tensorflow as tf

from tensorflow.keras import datasets, layers, models, Sequential
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import random
import numpy as np

In [2]:
fire_imgs = glob.glob("fire_dataset/fire_images/*")
non_fire_imgs = glob.glob("fire_dataset/non_fire_images/*")

In [3]:
fire_imgs

['fire_dataset/fire_images\\fire.1.png',
 'fire_dataset/fire_images\\fire.10.png',
 'fire_dataset/fire_images\\fire.100.png',
 'fire_dataset/fire_images\\fire.101.png',
 'fire_dataset/fire_images\\fire.102.png',
 'fire_dataset/fire_images\\fire.103.png',
 'fire_dataset/fire_images\\fire.104.png',
 'fire_dataset/fire_images\\fire.105.png',
 'fire_dataset/fire_images\\fire.106.png',
 'fire_dataset/fire_images\\fire.107.png',
 'fire_dataset/fire_images\\fire.108.png',
 'fire_dataset/fire_images\\fire.109.png',
 'fire_dataset/fire_images\\fire.11.png',
 'fire_dataset/fire_images\\fire.110.png',
 'fire_dataset/fire_images\\fire.111.png',
 'fire_dataset/fire_images\\fire.112.png',
 'fire_dataset/fire_images\\fire.113.png',
 'fire_dataset/fire_images\\fire.114.png',
 'fire_dataset/fire_images\\fire.115.png',
 'fire_dataset/fire_images\\fire.116.png',
 'fire_dataset/fire_images\\fire.117.png',
 'fire_dataset/fire_images\\fire.118.png',
 'fire_dataset/fire_images\\fire.119.png',
 'fire_dataset/

NameError: name 'df' is not defined

In [18]:
def load_img(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
    img = cv2.resize(img,(196,196))
    img = img / 255
    return img 

fire = pd.DataFrame([[i,1] for i in fire_imgs], 
                    columns=["url","label"])
non_fire = pd.DataFrame([[i,0] for i in non_fire_imgs], 
                        columns=["url","label"])
df = fire.append(non_fire)

In [19]:
df

,url,label
0,fire_dataset/fire_images\fire.1.png,1
1,fire_dataset/fire_images\fire.10.png,1
2,fire_dataset/fire_images\fire.100.png,1
3,fire_dataset/fire_images\fire.101.png,1
4,fire_dataset/fire_images\fire.102.png,1
...,...,...
238,fire_dataset/non_fire_images\non_fire.95.png,0
239,fire_dataset/non_fire_images\non_fire.96.png,0
240,fire_dataset/non_fire_images\non_fire.97.png,0
241,fire_dataset/non_fire_images\non_fire.98.png,0


In [20]:
X = []
y = []
for f,t in df.values:
    X.append(load_img(f))
    y.append(t)

X, y = np.array(X),np.array(y)

In [22]:
X.shape,y.shape

((998, 196, 196, 3), (998,))

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y, 
                                                    test_size=0.3, 
                                                    random_state=123, 
                                                    stratify=y)

In [28]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(196, 196, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Dense(1, activation='sigmoid'))


In [29]:
model.compile(optimizer='adam',
              loss="binary_crossentropy",
              metrics=['accuracy'])


In [30]:
model.fit(
          X_train,
          y_train,
          validation_split=0.3, 
          callbacks=[
                  tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                   patience=10,
                                                   restore_best_weights=True)
              ],
          epochs=50,
          batch_size=30
         )

Epoch 1/50
17/17 [==============================] - 23s 1s/step - loss: 2.8443 - accuracy: 0.5855 - val_loss: 0.2684 - val_accuracy: 0.8333
Epoch 2/50
17/17 [==============================] - 17s 1s/step - loss: 0.3408 - accuracy: 0.8440 - val_loss: 0.1602 - val_accuracy: 0.9524
Epoch 3/50
17/17 [==============================] - 18s 1s/step - loss: 0.1460 - accuracy: 0.9461 - val_loss: 0.1588 - val_accuracy: 0.9238
Epoch 4/50
17/17 [==============================] - 18s 1s/step - loss: 0.0957 - accuracy: 0.9622 - val_loss: 0.1364 - val_accuracy: 0.9429
Epoch 5/50
17/17 [==============================] - 19s 1s/step - loss: 0.0810 - accuracy: 0.9785 - val_loss: 0.1430 - val_accuracy: 0.9476
Epoch 6/50
17/17 [==============================] - 19s 1s/step - loss: 0.0870 - accuracy: 0.9679 - val_loss: 0.1199 - val_accuracy: 0.9476
Epoch 7/50
17/17 [==============================] - 17s 1s/step - loss: 0.0716 - accuracy: 0.9821 - val_loss: 0.1465 - val_accuracy: 0.9571
Epoch 8/50
17/17 [==

In [32]:
model.evaluate(X_test,y_test)

10/10 [==============================] - 2s 228ms/step - loss: 0.0791 - accuracy: 0.9700


[0.07907266914844513, 0.9700000286102295]

In [33]:
model.save("fire_cnn.h5")